# Data Science Capstone Project
## Title: Predicting Severity of Collisions

In [1]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


### Import libraries and the database:

In [2]:
import pandas as pd
import numpy as np
import itertools
import types
from botocore.client import Config
import ibm_boto3
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import matplotlib.ticker as ticker
%matplotlib inline

In [3]:
# The code was removed by Watson Studio for sharing.

### Conduct initial database loading and exploration:

In [4]:
cdf = pd.read_csv(body)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
cdf.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,5TH AVE NE AND NE 103RD ST,,NaN,2,Injury Collision,Angles,2,0,0,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Overcast,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,NaN,NaN,1,Property Damage Only Collision,Sideswipe,2,0,0,2,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),16,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",NaN,0,Raining,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,NaN,NaN,1,Property Damage Only Collision,Parked Car,4,0,0,3,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),14,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,0,Overcast,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,2ND AVE BETWEEN MARION ST AND MADISON ST,,NaN,1,Property Damage Only Collision,Other,3,0,0,3,2013/03/29 00:00:00+00,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,SWIFT AVE S AND SWIFT AV OFF RP,NaN,NaN,2,Injury Collision,Angles,2,0,0,2,2004/01/28 00:00:00+00,1/28/2004 8:04:00 AM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,0,Raining,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [5]:
cdf.shape

(194673, 38)

In [6]:
cdf["ROADCOND"].isnull().sum()

5012

In [7]:
cdf.groupby('SEVERITYCODE').size()

SEVERITYCODE
1    136485
2     58188
dtype: int64

In [8]:
cdf.columns.values

array(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY',
       'REPORTNO', 'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION',
       'EXCEPTRSNCODE', 'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC',
       'COLLISIONTYPE', 'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT',
       'VEHCOUNT', 'INCDATE', 'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE',
       'SDOT_COLDESC', 'INATTENTIONIND', 'UNDERINFL', 'WEATHER',
       'ROADCOND', 'LIGHTCOND', 'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING',
       'ST_COLCODE', 'ST_COLDESC', 'SEGLANEKEY', 'CROSSWALKKEY',
       'HITPARKEDCAR'], dtype=object)

### Create a subset of original database with features that seem to be relevant to predicting severity of collisions and verify data types:

In [9]:
cdf2 = cdf[['SEVERITYCODE', 'STATUS', 'ADDRTYPE', 'SEVERITYCODE.1', 'COLLISIONTYPE', 'PERSONCOUNT', 'PEDCOUNT', \
            'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE', 'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'INATTENTIONIND', \
            'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'PEDROWNOTGRNT', 'SPEEDING', 'ST_COLCODE', 'HITPARKEDCAR']]

In [10]:
cdf2.head()

,SEVERITYCODE,STATUS,ADDRTYPE,SEVERITYCODE.1,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLCODE,HITPARKEDCAR
0,2,Matched,Intersection,2,Angles,2,0,0,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,At Intersection (intersection related),11,NaN,N,Overcast,Wet,Daylight,NaN,NaN,10,N
1,1,Matched,Block,1,Sideswipe,2,0,0,2,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),16,NaN,0,Raining,Wet,Dark - Street Lights On,NaN,NaN,11,N
2,1,Matched,Block,1,Parked Car,4,0,0,3,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),14,NaN,0,Overcast,Dry,Daylight,NaN,NaN,32,N
3,1,Matched,Block,1,Other,3,0,0,3,2013/03/29 00:00:00+00,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),11,NaN,N,Clear,Dry,Daylight,NaN,NaN,23,N
4,2,Matched,Intersection,2,Angles,2,0,0,2,2004/01/28 00:00:00+00,1/28/2004 8:04:00 AM,At Intersection (intersection related),11,NaN,0,Raining,Wet,Daylight,NaN,NaN,10,N


In [11]:
cdf2.dtypes

SEVERITYCODE       int64
STATUS            object
ADDRTYPE          object
SEVERITYCODE.1     int64
COLLISIONTYPE     object
PERSONCOUNT        int64
PEDCOUNT           int64
PEDCYLCOUNT        int64
VEHCOUNT           int64
INCDATE           object
INCDTTM           object
JUNCTIONTYPE      object
SDOT_COLCODE       int64
INATTENTIONIND    object
UNDERINFL         object
WEATHER           object
ROADCOND          object
LIGHTCOND         object
PEDROWNOTGRNT     object
SPEEDING          object
ST_COLCODE        object
HITPARKEDCAR      object
dtype: object

### Delete unmatched records if they don't constitute a significant portion of the database:

In [12]:
cdf2.groupby('STATUS').size()

STATUS
Matched      189786
Unmatched      4887
dtype: int64

In [13]:
cdf2 = cdf2[cdf2.STATUS != "Unmatched"]
cdf2 = cdf2.drop('STATUS', 1)

### Delete duplicate dependent variabe, severity code, and transform values for classification:

In [14]:
cdf2.groupby('SEVERITYCODE.1').size()

SEVERITYCODE.1
1    132627
2     57159
dtype: int64

In [15]:
cdf2.groupby('SEVERITYCODE').size()

SEVERITYCODE
1    132627
2     57159
dtype: int64

In [16]:
cdf2 = cdf2.drop('SEVERITYCODE.1', 1)

In [17]:
cdf2['SEVERITYCODE'] = cdf2['SEVERITYCODE'].replace([1,2],[0,1])

### Transform time and date attributes to part of a day, weekend/weekday, and season attributes:

In [18]:
cdf2['INCDATE'] = pd.to_datetime(cdf2['INCDATE'])
cdf2['INCDTTM'] = pd.to_datetime(cdf2['INCDTTM'])

In [19]:
cdf2['period'] = (cdf2['INCDTTM'].dt.hour % 24 + 4) // 4
cdf2['period'].replace({1: 'Late Night',
                      2: 'Early Morning',
                      3: 'Morning',
                      4: 'Afternoon',
                      5: 'Evening',
                      6: 'Night'}, inplace=True)

In [20]:
cdf2 = cdf2.drop('INCDTTM', 1)

In [21]:
cdf2['dayofweek'] = cdf2['INCDATE'].dt.dayofweek
cdf2['weekend'] = cdf2['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)

In [22]:
cdf2['monthofyear'] = cdf2['INCDATE'].dt.month
cdf2['seasonofyear'] = (cdf2['monthofyear']%12 + 3)//3

cdf2['winter'] = cdf2['seasonofyear'].apply(lambda x: 1 if (x==1)  else 0)
cdf2['spring'] = cdf2['seasonofyear'].apply(lambda x: 1 if (x==2)  else 0)
cdf2['summer'] = cdf2['seasonofyear'].apply(lambda x: 1 if (x==3)  else 0)
cdf2['fall'] = cdf2['seasonofyear'].apply(lambda x: 1 if (x==4)  else 0)

In [23]:
cdf2 = cdf2.drop('dayofweek', 1)

cdf2 = cdf2.drop('monthofyear', 1)
cdf2 = cdf2.drop('seasonofyear', 1)

cdf2 = cdf2.drop('INCDATE', 1)

### Update and clean dummy variables in the database:

In [24]:
cdf2.groupby('INATTENTIONIND').size()

INATTENTIONIND
Y    29805
dtype: int64

In [25]:
cdf2['INATTENTIONIND'] = cdf2['INATTENTIONIND'].replace(np.nan, 0)
cdf2['INATTENTIONIND'] = cdf2['INATTENTIONIND'].replace("Y", 1)
cdf2.groupby('INATTENTIONIND').size()

INATTENTIONIND
0    159981
1     29805
dtype: int64

In [26]:
cdf2.groupby('UNDERINFL').size()

UNDERINFL
0     80391
1      3995
N    100274
Y      5126
dtype: int64

In [27]:
cdf2['UNDERINFL'] = cdf2['UNDERINFL'].replace(['0','N'],0)
cdf2['UNDERINFL'] = cdf2['UNDERINFL'].replace(['1','Y'],1)
cdf2.groupby('UNDERINFL').size()

UNDERINFL
0    180665
1      9121
dtype: int64

In [28]:
cdf2.groupby('PEDROWNOTGRNT').size()

PEDROWNOTGRNT
Y    4667
dtype: int64

In [29]:
cdf2['PEDROWNOTGRNT'] = cdf2['PEDROWNOTGRNT'].replace(np.nan, 0)
cdf2['PEDROWNOTGRNT'] = cdf2['PEDROWNOTGRNT'].replace("Y", 1)
cdf2.groupby('PEDROWNOTGRNT').size()

PEDROWNOTGRNT
0    185119
1      4667
dtype: int64

In [30]:
cdf2.groupby('SPEEDING').size()

SPEEDING
Y    9333
dtype: int64

In [31]:
cdf2['SPEEDING'] = cdf2['SPEEDING'].replace(np.nan, 0)
cdf2['SPEEDING'] = cdf2['SPEEDING'].replace("Y", 1)
cdf2.groupby('SPEEDING').size()

SPEEDING
0    180453
1      9333
dtype: int64

In [32]:
cdf2.groupby('HITPARKEDCAR').size()

HITPARKEDCAR
N    182868
Y      6918
dtype: int64

In [33]:
cdf2['HITPARKEDCAR'] = cdf2['HITPARKEDCAR'].replace("N", 0)
cdf2['HITPARKEDCAR'] = cdf2['HITPARKEDCAR'].replace("Y", 1)
cdf2.groupby('HITPARKEDCAR').size()

HITPARKEDCAR
0    182868
1      6918
dtype: int64

In [34]:
cdf2.head()

,SEVERITYCODE,ADDRTYPE,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,SDOT_COLCODE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLCODE,HITPARKEDCAR,period,weekend,winter,spring,summer,fall
0,1,Intersection,Angles,2,0,0,2,At Intersection (intersection related),11,0,0,Overcast,Wet,Daylight,0,0,10,0,Afternoon,0,0,1,0,0
1,0,Block,Sideswipe,2,0,0,2,Mid-Block (not related to intersection),16,0,0,Raining,Wet,Dark - Street Lights On,0,0,11,0,Evening,0,1,0,0,0
2,0,Block,Parked Car,4,0,0,3,Mid-Block (not related to intersection),14,0,0,Overcast,Dry,Daylight,0,0,32,0,Morning,0,0,0,0,1
3,0,Block,Other,3,0,0,3,Mid-Block (not related to intersection),11,0,0,Clear,Dry,Daylight,0,0,23,0,Morning,1,0,1,0,0
4,1,Intersection,Angles,2,0,0,2,At Intersection (intersection related),11,0,0,Raining,Wet,Daylight,0,0,10,0,Morning,0,1,0,0,0


### Evaluate potential impact of selected categorical attributes on severity of collisions before transforming them into dummy variables:

In [35]:
cdf2.groupby('ADDRTYPE').size()

ADDRTYPE
Alley              747
Block           123663
Intersection     63559
dtype: int64

In [36]:
cdf2.groupby(['ADDRTYPE'])['SEVERITYCODE'].value_counts(normalize=True)

ADDRTYPE      SEVERITYCODE
Alley         0               0.891566
              1               0.108434
Block         0               0.761473
              1               0.238527
Intersection  0               0.568731
              1               0.431269
Name: SEVERITYCODE, dtype: float64

In [37]:
cdf2.groupby(['SEVERITYCODE'])['ADDRTYPE'].value_counts(normalize=True)

SEVERITYCODE  ADDRTYPE    
0             Block           0.718934
              Intersection    0.275981
              Alley           0.005085
1             Block           0.517591
              Intersection    0.480988
              Alley           0.001421
Name: ADDRTYPE, dtype: float64

In [38]:
cdf2.groupby('COLLISIONTYPE').size()

COLLISIONTYPE
Angles        34674
Cycles         5415
Head On        2024
Left Turn     13703
Other         23703
Parked Car    47986
Pedestrian     6607
Rear Ended    34089
Right Turn     2956
Sideswipe     18608
dtype: int64

In [39]:
cdf2.groupby(['COLLISIONTYPE'])['SEVERITYCODE'].value_counts(normalize=True)

COLLISIONTYPE  SEVERITYCODE
Angles         0               0.607083
               1               0.392917
Cycles         1               0.876085
               0               0.123915
Head On        0               0.569170
               1               0.430830
Left Turn      0               0.605123
               1               0.394877
Other          0               0.742142
               1               0.257858
Parked Car     0               0.944525
               1               0.055475
Pedestrian     1               0.898290
               0               0.101710
Rear Ended     0               0.569627
               1               0.430373
Right Turn     0               0.793978
               1               0.206022
Sideswipe      0               0.865327
               1               0.134673
Name: SEVERITYCODE, dtype: float64

In [40]:
cdf2.groupby(['SEVERITYCODE'])['COLLISIONTYPE'].value_counts(normalize=True)

SEVERITYCODE  COLLISIONTYPE
0             Parked Car       0.341761
              Angles           0.158725
              Rear Ended       0.146419
              Other            0.132643
              Sideswipe        0.121415
              Left Turn        0.062525
              Right Turn       0.017697
              Head On          0.008687
              Pedestrian       0.005067
              Cycles           0.005060
1             Rear Ended       0.256728
              Angles           0.238407
              Other            0.106954
              Pedestrian       0.103857
              Left Turn        0.094687
              Cycles           0.083015
              Parked Car       0.046582
              Sideswipe        0.043853
              Head On          0.015259
              Right Turn       0.010657
Name: COLLISIONTYPE, dtype: float64

In [41]:
cdf2.groupby('JUNCTIONTYPE').size()

JUNCTIONTYPE
At Intersection (but not related to intersection)     2060
At Intersection (intersection related)               61334
Driveway Junction                                    10538
Mid-Block (but intersection related)                 22388
Mid-Block (not related to intersection)              87119
Ramp Junction                                          162
Unknown                                                  7
dtype: int64

In [42]:
cdf2.groupby(['JUNCTIONTYPE'])['SEVERITYCODE'].value_counts(normalize=True)

JUNCTIONTYPE                                       SEVERITYCODE
At Intersection (but not related to intersection)  0               0.700971
                                                   1               0.299029
At Intersection (intersection related)             0               0.563505
                                                   1               0.436495
Driveway Junction                                  0               0.696242
                                                   1               0.303758
Mid-Block (but intersection related)               0               0.678176
                                                   1               0.321824
Mid-Block (not related to intersection)            0               0.782757
                                                   1               0.217243
Ramp Junction                                      0               0.679012
                                                   1               0.320988
Unknown                 

In [43]:
cdf2.groupby(['SEVERITYCODE'])['JUNCTIONTYPE'].value_counts(normalize=True)

SEVERITYCODE  JUNCTIONTYPE                                     
0             Mid-Block (not related to intersection)              0.537651
              At Intersection (intersection related)               0.272496
              Mid-Block (but intersection related)                 0.119707
              Driveway Junction                                    0.057847
              At Intersection (but not related to intersection)    0.011385
              Ramp Junction                                        0.000867
              Unknown                                              0.000047
1             At Intersection (intersection related)               0.471562
              Mid-Block (not related to intersection)              0.333363
              Mid-Block (but intersection related)                 0.126909
              Driveway Junction                                    0.056382
              At Intersection (but not related to intersection)    0.010850
              Ramp Junct

In [44]:
cdf2.groupby('WEATHER').size()

WEATHER
Blowing Sand/Dirt               56
Clear                       111134
Fog/Smog/Smoke                 569
Other                          832
Overcast                     27713
Partly Cloudy                    5
Raining                      33144
Severe Crosswind                25
Sleet/Hail/Freezing Rain       113
Snowing                        907
Unknown                      15091
dtype: int64

In [45]:
cdf2.groupby(['WEATHER'])['SEVERITYCODE'].value_counts(normalize=True)

WEATHER                   SEVERITYCODE
Blowing Sand/Dirt         0               0.732143
                          1               0.267857
Clear                     0               0.677506
                          1               0.322494
Fog/Smog/Smoke            0               0.671353
                          1               0.328647
Other                     0               0.860577
                          1               0.139423
Overcast                  0               0.684444
                          1               0.315556
Partly Cloudy             1               0.600000
                          0               0.400000
Raining                   0               0.662805
                          1               0.337195
Severe Crosswind          0               0.720000
                          1               0.280000
Sleet/Hail/Freezing Rain  0               0.752212
                          1               0.247788
Snowing                   0               0

In [46]:
cdf2.groupby(['SEVERITYCODE'])['WEATHER'].value_counts(normalize=True)

SEVERITYCODE  WEATHER                 
0             Clear                       0.568321
              Raining                     0.165815
              Overcast                    0.143171
              Unknown                     0.107748
              Snowing                     0.005555
              Other                       0.005404
              Fog/Smog/Smoke              0.002883
              Sleet/Hail/Freezing Rain    0.000642
              Blowing Sand/Dirt           0.000309
              Severe Crosswind            0.000136
              Partly Cloudy               0.000015
1             Clear                       0.627627
              Raining                     0.195713
              Overcast                    0.153142
              Unknown                     0.014290
              Fog/Smog/Smoke              0.003275
              Snowing                     0.002995
              Other                       0.002031
              Sleet/Hail/Freezing Rain    0

In [47]:
cdf2.groupby('ROADCOND').size()

ROADCOND
Dry               124508
Ice                 1209
Oil                   64
Other                132
Sand/Mud/Dirt         75
Snow/Slush          1004
Standing Water       115
Unknown            15078
Wet                47473
dtype: int64

In [48]:
cdf2.groupby(['ROADCOND'])['SEVERITYCODE'].value_counts(normalize=True)

ROADCOND        SEVERITYCODE
Dry             0               0.678221
                1               0.321779
Ice             0               0.774194
                1               0.225806
Oil             0               0.625000
                1               0.375000
Other           0               0.674242
                1               0.325758
Sand/Mud/Dirt   0               0.693333
                1               0.306667
Snow/Slush      0               0.833665
                1               0.166335
Standing Water  0               0.739130
                1               0.260870
Unknown         0               0.950325
                1               0.049675
Wet             0               0.668127
                1               0.331873
Name: SEVERITYCODE, dtype: float64

In [49]:
cdf2.groupby(['SEVERITYCODE'])['ROADCOND'].value_counts(normalize=True)

SEVERITYCODE  ROADCOND      
0             Dry               0.637169
              Wet               0.239327
              Unknown           0.108119
              Ice               0.007063
              Snow/Slush        0.006316
              Other             0.000672
              Standing Water    0.000641
              Sand/Mud/Dirt     0.000392
              Oil               0.000302
1             Dry               0.701302
              Wet               0.275784
              Unknown           0.013111
              Ice               0.004779
              Snow/Slush        0.002923
              Other             0.000753
              Standing Water    0.000525
              Oil               0.000420
              Sand/Mud/Dirt     0.000403
Name: ROADCOND, dtype: float64

In [50]:
cdf2.groupby('LIGHTCOND').size()

LIGHTCOND
Dark - No Street Lights       1537
Dark - Street Lights Off      1199
Dark - Street Lights On      48506
Dark - Unknown Lighting         11
Dawn                          2502
Daylight                    116135
Dusk                          5902
Other                          235
Unknown                      13473
dtype: int64

In [51]:
cdf2.groupby(['LIGHTCOND'])['SEVERITYCODE'].value_counts(normalize=True)

LIGHTCOND                 SEVERITYCODE
Dark - No Street Lights   0               0.782694
                          1               0.217306
Dark - Street Lights Off  0               0.736447
                          1               0.263553
Dark - Street Lights On   0               0.701583
                          1               0.298417
Dark - Unknown Lighting   0               0.636364
                          1               0.363636
Dawn                      0               0.670663
                          1               0.329337
Daylight                  0               0.668110
                          1               0.331890
Dusk                      0               0.670620
                          1               0.329380
Other                     0               0.778723
                          1               0.221277
Unknown                   0               0.955095
                          1               0.044905
Name: SEVERITYCODE, dtype: float64

In [52]:
cdf2.groupby(['SEVERITYCODE'])['LIGHTCOND'].value_counts(normalize=True)

SEVERITYCODE  LIGHTCOND               
0             Daylight                    0.586026
              Dark - Street Lights On     0.257028
              Unknown                     0.097189
              Dusk                        0.029894
              Dawn                        0.012674
              Dark - No Street Lights     0.009086
              Dark - Street Lights Off    0.006669
              Other                       0.001382
              Dark - Unknown Lighting     0.000053
1             Daylight                    0.675050
              Dark - Street Lights On     0.253512
              Dusk                        0.034047
              Dawn                        0.014431
              Unknown                     0.010596
              Dark - No Street Lights     0.005850
              Dark - Street Lights Off    0.005534
              Other                       0.000911
              Dark - Unknown Lighting     0.000070
Name: LIGHTCOND, dtype: float64

### Evaluate and handle missing values:

In [53]:
 cdf2.isnull().sum()

SEVERITYCODE         0
ADDRTYPE          1817
COLLISIONTYPE       21
PERSONCOUNT          0
PEDCOUNT             0
PEDCYLCOUNT          0
VEHCOUNT             0
JUNCTIONTYPE      6178
SDOT_COLCODE         0
INATTENTIONIND       0
UNDERINFL            0
WEATHER            197
ROADCOND           128
LIGHTCOND          286
PEDROWNOTGRNT        0
SPEEDING             0
ST_COLCODE          18
HITPARKEDCAR         0
period               0
weekend              0
winter               0
spring               0
summer               0
fall                 0
dtype: int64

#### "NOT ENOUGH INFORMATION / NOT APPLICABLE" for SDOT_COLDESC attribute for records with missing JUNCTIONTYPE so those rows is better to be deleted instead of generate the missing values:

In [54]:
cdf2[cdf2['JUNCTIONTYPE'].isnull()].head()

,SEVERITYCODE,ADDRTYPE,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,SDOT_COLCODE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLCODE,HITPARKEDCAR,period,weekend,winter,spring,summer,fall
103,0,Block,Parked Car,2,0,0,2,NaN,0,0,0,Clear,Dry,Daylight,0,0,32,1,Evening,0,0,1,0,0
127,0,Block,Parked Car,2,0,0,2,NaN,0,0,0,Clear,Dry,Daylight,0,0,32,0,Morning,1,1,0,0,0
160,1,Block,Rear Ended,2,0,0,2,NaN,14,1,0,Overcast,Dry,Daylight,0,0,13,0,Early Morning,1,0,0,1,0
171,0,NaN,Parked Car,2,0,0,2,NaN,0,0,0,Clear,Dry,Daylight,0,0,32,0,Evening,1,0,0,1,0
261,0,Alley,Other,1,0,0,1,NaN,0,0,0,Unknown,Unknown,Unknown,0,0,50,0,Late Night,1,0,0,1,0


In [55]:
cdf2 = cdf2.dropna(axis = 0, subset=['JUNCTIONTYPE'])

#### Delete rows with missing values for important attributes, COLLISIONTYPE and ST_COLCODE:

In [56]:
cdf2 = cdf2.dropna(axis = 0, subset=['COLLISIONTYPE'])
cdf2 = cdf2.dropna(axis = 0, subset=['ST_COLCODE'])

#### Evaluate the importance of records with missing values for ADDRTYPE, WEATHER, ROADCOND, and LIGHTCOND and delete them if values for severity code (dependent variable) are evently distributed for such records:

In [57]:
cdf2[['ADDRTYPE', 'WEATHER', 'ROADCOND', 'LIGHTCOND']] = cdf2[['ADDRTYPE', 'WEATHER', 'ROADCOND', 'LIGHTCOND']].fillna(value='Null')

In [58]:
cdf2[cdf2['ADDRTYPE'] == 'Null'].head()

,SEVERITYCODE,ADDRTYPE,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,SDOT_COLCODE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLCODE,HITPARKEDCAR,period,weekend,winter,spring,summer,fall
268,0,Null,Parked Car,3,0,0,3,Mid-Block (not related to intersection),14,0,0,Clear,Dry,Dark - Street Lights On,0,0,32,0,Late Night,1,0,1,0,0
412,0,Null,Left Turn,2,0,0,2,Unknown,0,0,0,Clear,Dry,Daylight,0,0,28,0,Evening,1,0,0,1,0
4421,0,Null,Parked Car,2,0,0,2,Mid-Block (but intersection related),13,0,0,Unknown,Unknown,Daylight,0,0,32,0,Evening,1,0,0,0,1
5145,0,Null,Parked Car,2,0,0,2,Mid-Block (not related to intersection),0,0,0,Clear,Dry,Daylight,0,0,32,0,Evening,0,0,0,1,0
6079,0,Null,Sideswipe,2,0,0,2,Mid-Block (not related to intersection),16,0,0,Clear,Dry,Daylight,0,0,26,0,Evening,0,0,0,1,0


In [59]:
cdf2.groupby(['ADDRTYPE'])['SEVERITYCODE'].value_counts(normalize=True)

ADDRTYPE      SEVERITYCODE
Alley         0               0.876596
              1               0.123404
Block         0               0.755026
              1               0.244974
Intersection  0               0.568089
              1               0.431911
Null          0               0.886121
              1               0.113879
Name: SEVERITYCODE, dtype: float64

In [60]:
cdf2.groupby(['SEVERITYCODE'])['ADDRTYPE'].value_counts(normalize=True)

SEVERITYCODE  ADDRTYPE    
0             Block           0.712455
              Intersection    0.283957
              Null            0.001963
              Alley           0.001624
1             Block           0.516526
              Intersection    0.482400
              Null            0.000564
              Alley           0.000511
Name: ADDRTYPE, dtype: float64

##### Distribution of dependent variable for Null ADDRTYPE is very similar to those for Alley. Replacing Null with Alley for the dataset:

In [61]:
cdf2['ADDRTYPE'] = cdf2['ADDRTYPE'].replace(['Null'],'Alley')

In [62]:
cdf2[cdf2['WEATHER'] == 'Null'].head()

,SEVERITYCODE,ADDRTYPE,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,SDOT_COLCODE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLCODE,HITPARKEDCAR,period,weekend,winter,spring,summer,fall
486,0,Block,Parked Car,2,0,0,2,Driveway Junction,11,0,0,Null,Unknown,Unknown,0,0,32,1,Morning,0,0,1,0,0
490,0,Block,Parked Car,2,0,0,2,Mid-Block (not related to intersection),11,0,0,Null,Null,Null,0,0,19,0,Morning,0,0,0,1,0
1673,1,Intersection,Pedestrian,2,1,0,1,At Intersection (intersection related),24,0,0,Null,Null,Null,0,0,0,0,Evening,1,0,0,1,0
1922,0,Block,Parked Car,2,0,0,2,Mid-Block (not related to intersection),12,1,0,Null,Dry,Daylight,0,0,32,0,Afternoon,1,0,0,1,0
2152,1,Block,Pedestrian,2,1,0,1,Mid-Block (not related to intersection),24,0,0,Null,Null,Null,0,0,0,0,Late Night,1,0,1,0,0


##### It seems records with missing values for WEATHER are also missing values for ROADCOND and LIGHTCOND and deleting them could be practical but first further explore those records:

In [63]:
cdf2.groupby(['WEATHER'])['SEVERITYCODE'].value_counts(normalize=True)

WEATHER                   SEVERITYCODE
Blowing Sand/Dirt         0               0.740000
                          1               0.260000
Clear                     0               0.673944
                          1               0.326056
Fog/Smog/Smoke            0               0.664875
                          1               0.335125
Null                      0               0.715847
                          1               0.284153
Other                     0               0.848806
                          1               0.151194
Overcast                  0               0.681062
                          1               0.318938
Partly Cloudy             1               0.600000
                          0               0.400000
Raining                   0               0.660683
                          1               0.339317
Severe Crosswind          0               0.720000
                          1               0.280000
Sleet/Hail/Freezing Rain  0               0

In [64]:
cdf2.groupby(['SEVERITYCODE'])['WEATHER'].value_counts(normalize=True)

SEVERITYCODE  WEATHER                 
0             Clear                       0.580703
              Raining                     0.170332
              Overcast                    0.146228
              Unknown                     0.086944
              Snowing                     0.005669
              Other                       0.005046
              Fog/Smog/Smoke              0.002925
              Null                        0.001033
              Sleet/Hail/Freezing Rain    0.000670
              Blowing Sand/Dirt           0.000292
              Severe Crosswind            0.000142
              Partly Cloudy               0.000016
1             Clear                       0.627766
              Raining                     0.195472
              Overcast                    0.153013
              Unknown                     0.013672
              Fog/Smog/Smoke              0.003295
              Snowing                     0.002977
              Other                       0

##### The distribution of values for severity code is fairly even for Null values of WEATHER attribute. We can safely delete these records since this is most likely not a systematic signal biased toward a specific outcome:

In [65]:
cdf2 = cdf2[cdf2.WEATHER != "Null"]

##### Next, evaluate the distribution of values for severity code for records with Null values for ROADCOND attribute and similarly delete those records if severity code values are evenly distributed:

In [66]:
cdf2.groupby(['ROADCOND'])['SEVERITYCODE'].value_counts(normalize=True)

ROADCOND        SEVERITYCODE
Dry             0               0.674892
                1               0.325108
Ice             0               0.773345
                1               0.226655
Null            0               0.733333
                1               0.266667
Oil             0               0.600000
                1               0.400000
Other           0               0.658537
                1               0.341463
Sand/Mud/Dirt   0               0.671642
                1               0.328358
Snow/Slush      0               0.831976
                1               0.168024
Standing Water  0               0.724771
                1               0.275229
Unknown         0               0.939277
                1               0.060723
Wet             0               0.665491
                1               0.334509
Name: SEVERITYCODE, dtype: float64

In [67]:
cdf2.groupby(['SEVERITYCODE'])['ROADCOND'].value_counts(normalize=True)

SEVERITYCODE  ROADCOND      
0             Dry               0.651928
              Wet               0.245710
              Unknown           0.086560
              Ice               0.007190
              Snow/Slush        0.006448
              Other             0.000639
              Standing Water    0.000624
              Sand/Mud/Dirt     0.000355
              Oil               0.000284
              Null              0.000260
1             Dry               0.701647
              Wet               0.275940
              Unknown           0.012503
              Ice               0.004708
              Snow/Slush        0.002910
              Other             0.000741
              Standing Water    0.000529
              Oil               0.000423
              Sand/Mud/Dirt     0.000388
              Null              0.000212
Name: ROADCOND, dtype: float64

In [68]:
cdf2 = cdf2[cdf2.ROADCOND != "Null"]

##### Lastly, evaluate the distribution of values for severity code for records with Null values for LIGHTCOND attribute and similarly delete those records if severity code values are evenly distributed:

In [69]:
cdf2.groupby(['LIGHTCOND'])['SEVERITYCODE'].value_counts(normalize=True)

LIGHTCOND                 SEVERITYCODE
Dark - No Street Lights   0               0.775650
                          1               0.224350
Dark - Street Lights Off  0               0.729706
                          1               0.270294
Dark - Street Lights On   0               0.698250
                          1               0.301750
Dark - Unknown Lighting   0               0.636364
                          1               0.363636
Dawn                      0               0.666123
                          1               0.333877
Daylight                  0               0.664101
                          1               0.335899
Dusk                      0               0.666782
                          1               0.333218
Null                      0               0.786096
                          1               0.213904
Other                     0               0.753555
                          1               0.246445
Unknown                   0               0

In [70]:
cdf2.groupby(['SEVERITYCODE'])['LIGHTCOND'].value_counts(normalize=True)

SEVERITYCODE  LIGHTCOND               
0             Daylight                    0.597474
              Dark - Street Lights On     0.262338
              Unknown                     0.078767
              Dusk                        0.030427
              Dawn                        0.012900
              Dark - No Street Lights     0.008953
              Dark - Street Lights Off    0.006671
              Other                       0.001255
              Null                        0.001161
              Dark - Unknown Lighting     0.000055
1             Daylight                    0.675145
              Dark - Street Lights On     0.253281
              Dusk                        0.033971
              Dawn                        0.014445
              Unknown                     0.010159
              Dark - No Street Lights     0.005785
              Dark - Street Lights Off    0.005521
              Other                       0.000917
              Null                        0

In [71]:
cdf2 = cdf2[cdf2.LIGHTCOND != "Null"]

In [72]:
cdf2.isnull().sum()

SEVERITYCODE      0
ADDRTYPE          0
COLLISIONTYPE     0
PERSONCOUNT       0
PEDCOUNT          0
PEDCYLCOUNT       0
VEHCOUNT          0
JUNCTIONTYPE      0
SDOT_COLCODE      0
INATTENTIONIND    0
UNDERINFL         0
WEATHER           0
ROADCOND          0
LIGHTCOND         0
PEDROWNOTGRNT     0
SPEEDING          0
ST_COLCODE        0
HITPARKEDCAR      0
period            0
weekend           0
winter            0
spring            0
summer            0
fall              0
dtype: int64

##### Deleting rows with missing values for WEATHER also eliminates the rows with missing values for ROADCOND and LIGHTCOND.

In [73]:
cdf2.head()

,SEVERITYCODE,ADDRTYPE,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,SDOT_COLCODE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLCODE,HITPARKEDCAR,period,weekend,winter,spring,summer,fall
0,1,Intersection,Angles,2,0,0,2,At Intersection (intersection related),11,0,0,Overcast,Wet,Daylight,0,0,10,0,Afternoon,0,0,1,0,0
1,0,Block,Sideswipe,2,0,0,2,Mid-Block (not related to intersection),16,0,0,Raining,Wet,Dark - Street Lights On,0,0,11,0,Evening,0,1,0,0,0
2,0,Block,Parked Car,4,0,0,3,Mid-Block (not related to intersection),14,0,0,Overcast,Dry,Daylight,0,0,32,0,Morning,0,0,0,0,1
3,0,Block,Other,3,0,0,3,Mid-Block (not related to intersection),11,0,0,Clear,Dry,Daylight,0,0,23,0,Morning,1,0,1,0,0
4,1,Intersection,Angles,2,0,0,2,At Intersection (intersection related),11,0,0,Raining,Wet,Daylight,0,0,10,0,Morning,0,1,0,0,0


In [74]:
cdf2.columns.values

array(['SEVERITYCODE', 'ADDRTYPE', 'COLLISIONTYPE', 'PERSONCOUNT',
       'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'JUNCTIONTYPE',
       'SDOT_COLCODE', 'INATTENTIONIND', 'UNDERINFL', 'WEATHER',
       'ROADCOND', 'LIGHTCOND', 'PEDROWNOTGRNT', 'SPEEDING', 'ST_COLCODE',
       'HITPARKEDCAR', 'period', 'weekend', 'winter', 'spring', 'summer',
       'fall'], dtype=object)

### Address imbalanced dataset before separating independent variables from the dependent variable by downsampling majority class:

In [75]:
cdf2.groupby('SEVERITYCODE').size()

SEVERITYCODE
0    126518
1     56656
dtype: int64

In [76]:
from sklearn.utils import resample

# Separate majority and minority classes
cdf_majority = cdf2[cdf2.SEVERITYCODE == 0]
cdf_minority = cdf2[cdf2.SEVERITYCODE == 1]

In [77]:
cdf_minority.head()

,SEVERITYCODE,ADDRTYPE,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,JUNCTIONTYPE,SDOT_COLCODE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLCODE,HITPARKEDCAR,period,weekend,winter,spring,summer,fall
0,1,Intersection,Angles,2,0,0,2,At Intersection (intersection related),11,0,0,Overcast,Wet,Daylight,0,0,10,0,Afternoon,0,0,1,0,0
4,1,Intersection,Angles,2,0,0,2,At Intersection (intersection related),11,0,0,Raining,Wet,Daylight,0,0,10,0,Morning,0,1,0,0,0
7,1,Intersection,Cycles,3,0,1,1,At Intersection (intersection related),51,0,0,Clear,Dry,Daylight,0,0,5,0,Evening,0,0,1,0,0
9,1,Intersection,Angles,2,0,0,2,At Intersection (intersection related),11,0,0,Clear,Dry,Daylight,0,0,10,0,Afternoon,0,0,1,0,0
14,1,Block,Head On,2,0,0,2,Mid-Block (not related to intersection),11,0,0,Clear,Dry,Dark - Street Lights On,0,0,30,0,Late Night,1,0,1,0,0


In [78]:
# Downsample majority class
cdf_majority_downsampled = resample(cdf_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=56656,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
cdf3 = pd.concat([cdf_majority_downsampled, cdf_minority])
 
# Display new class counts
cdf3.SEVERITYCODE.value_counts()

1    56656
0    56656
Name: SEVERITYCODE, dtype: int64

### Separate dependent variable from the independent variables 

#### Create two subsets of independent variables and normalize them:

##### Subset 1 (X1) - All relevant indendent variables:

In [79]:
Feature1 = cdf3[['PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INATTENTIONIND', 'UNDERINFL', \
                'PEDROWNOTGRNT', 'SPEEDING', 'HITPARKEDCAR', 'weekend', 'winter', 'spring', 'summer']]

In [80]:
Feature1 = pd.concat([Feature1, pd.get_dummies(cdf3[['ADDRTYPE', 'COLLISIONTYPE', 'JUNCTIONTYPE', 'WEATHER', \
                    'ROADCOND', 'LIGHTCOND', 'period']], prefix='', prefix_sep='')], axis = 1)
Feature1 = pd.concat([Feature1, pd.get_dummies(cdf3['SDOT_COLCODE'], prefix='SDOTCODE', prefix_sep='.')], axis = 1)
Feature1 = pd.concat([Feature1, pd.get_dummies(cdf3[['ST_COLCODE']], prefix='STCODE', prefix_sep='.')], axis = 1)

In [81]:
X1 = preprocessing.StandardScaler().fit(Feature1).transform(Feature1)
X1[0:5]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':


array([[ 0.32125901, -0.23940111, -0.21513193, ..., -0.01706799,
        -0.00297074, -0.00727695],
       [-0.38335553, -0.23940111, -0.21513193, ..., -0.01706799,
        -0.00297074, -0.00727695],
       [-0.38335553, -0.23940111, -0.21513193, ..., -0.01706799,
        -0.00297074, -0.00727695],
       [ 0.32125901, -0.23940111, -0.21513193, ..., -0.01706799,
        -0.00297074, -0.00727695],
       [-0.38335553, -0.23940111, -0.21513193, ..., -0.01706799,
        -0.00297074, -0.00727695]])

##### Subset 2 (X2) - Independent variables easily identifiable before a collision:

In [82]:
Feature2 = cdf3[['INATTENTIONIND', 'UNDERINFL', 'PEDROWNOTGRNT', 'SPEEDING', 'HITPARKEDCAR', 'weekend', 'winter', 'spring', 'summer']]

In [83]:
Feature2 = pd.concat([Feature2, pd.get_dummies(cdf3[['ADDRTYPE', 'JUNCTIONTYPE', 'WEATHER', \
                    'ROADCOND', 'LIGHTCOND', 'period']], prefix='', prefix_sep='')], axis = 1)

In [84]:
X2 = preprocessing.StandardScaler().fit(Feature2).transform(Feature2)
X2[0:5]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':


array([[-4.45872608e-01, -2.37266980e-01, -2.01044011e-01,
         4.23096729e+00, -1.59756721e-01, -8.47498867e-01,
         1.82175367e+00, -5.81141158e-01, -5.85611749e-01,
        -4.85079832e-02,  7.91431319e-01, -7.87513728e-01,
        -1.07100182e-01, -7.68614380e-01, -2.47190095e-01,
         2.67826627e+00, -8.79926318e-01, -3.07439089e-02,
        -4.20127276e-03, -1.70679856e-02, -1.24014244e+00,
        -5.66907389e-02, -5.91451152e-02, -4.19388854e-01,
        -5.14550997e-03,  2.11237673e+00, -1.07117058e-02,
        -2.45045570e-02, -6.44687292e-02, -2.28206925e-01,
        -1.45031654e+00, -7.70077083e-02, -1.90253459e-02,
        -2.77196959e-02, -1.87918143e-02, -6.69744504e-02,
        -2.45045570e-02, -2.26231036e-01,  1.68227495e+00,
        -8.70403759e-02, -7.91246614e-02, -5.89852007e-01,
        -7.27694632e-03, -1.18992158e-01,  7.54554885e-01,
        -1.81818596e-01, -3.26954062e-02, -2.14157885e-01,
        -5.42152715e-01, -2.67751171e-01, -5.44101033e-0

#### Create a subset for the dependent variable:

In [85]:
y = cdf3['SEVERITYCODE'].values
y[0:5]

array([0, 0, 0, 0, 0], dtype=int64)

## MODELING

### Test/Train partitioning

##### Partitioning for Subset 1 of independent variables (X1):

In [86]:
from sklearn.model_selection import train_test_split
X1_train, X1_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=4)
print ('Train set:', X1_train.shape,  y_train.shape)
print ('Test set:', X1_test.shape,  y_test.shape)

Train set: (90649, 217) (90649,)
Test set: (22663, 217) (22663,)


##### Partitioning for Subset 2 of independent variables (X2):

In [87]:
from sklearn.model_selection import train_test_split
X2_train, X2_test, y_train, y_test = train_test_split(X2, y, test_size=0.2, random_state=4)
print ('Train set:', X2_train.shape,  y_train.shape)
print ('Test set:', X2_test.shape,  y_test.shape)

Train set: (90649, 54) (90649,)
Test set: (22663, 54) (22663,)


### K Nearest Neighbor(KNN)

In [95]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [119]:
## Best k
#Ks=15
#mean_acc1=np.zeros((Ks-1))
#std_acc1=np.zeros((Ks-1))
#ConfustionMx1=[];
#for n in range(1,Ks):
    
    #Train Model and Predict  
#    kNN_model1 = KNeighborsClassifier(n_neighbors=n).fit(X1_train,y_train)
#    yhat = kNN_model1.predict(X1_test)
    
    
#    mean_acc1[n-1]=np.mean(yhat==y_test);
    
#    std_acc1[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])
#mean_acc1

In [120]:
#print("The highest accuracy is ", mean_acc1.max(), "with k=", mean_acc1.argmax()+1)
print("The highest accuracy is ", 0.6920090014561179, "with k=", 13)

The highest accuracy is  0.6920090014561179 with k= 13


In [ ]:
k = 13
kNN_model1 = KNeighborsClassifier(n_neighbors = k).fit(X1_train,y_train)
kNN_model1

In [96]:
# Best k
Ks=15
mean_acc2=np.zeros((Ks-1))
std_acc2=np.zeros((Ks-1))
ConfustionMx2=[];
for n in range(1,Ks):
    
    #Train Model and Predict  
    kNN_model2 = KNeighborsClassifier(n_neighbors=n).fit(X2_train,y_train)
    yhat = kNN_model2.predict(X2_test)
    
    
    mean_acc2[n-1]=np.mean(yhat==y_test);
    
    std_acc2[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])
mean_acc2

array([0.56029652, 0.55186868, 0.57326921, 0.57101884, 0.58178529,
       0.57781406, 0.58509465, 0.57979967, 0.58919825, 0.58760976,
       0.59493447, 0.59043375, 0.59537572, 0.59303711])

In [97]:
print("The highest accuracy is", mean_acc2.max(), "with k=", mean_acc2.argmax()+1)

The highest accuracy is 0.5953757225433526 with k= 13


In [98]:
kNN_model2 = KNeighborsClassifier(n_neighbors = mean_acc2.argmax()+1).fit(X2_train,y_train)
kNN_model2

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=13, p=2,
           weights='uniform')

### Decision Tree

In [88]:
from sklearn.tree import DecisionTreeClassifier

In [114]:
DT_model1 = DecisionTreeClassifier(criterion="entropy", max_depth = 4).fit(X1_train,y_train)
DT_model1

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [115]:
DT_model2 = DecisionTreeClassifier(criterion="entropy", max_depth = 4).fit(X2_train,y_train)
DT_model2

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

### Support Vector Machine

In [99]:
from sklearn import svm

In [104]:
SVM_model1 = svm.SVC().fit(X1_train, y_train)
SVM_model1

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [100]:
SVM_model2 = svm.SVC().fit(X2_train, y_train)
SVM_model2

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

### Logistic Regression

In [101]:
from sklearn.linear_model import LogisticRegression

In [102]:
LR_model1 = LogisticRegression(C=0.01).fit(X1_train,y_train)
LR_model1

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [103]:
LR_model2 = LogisticRegression(C=0.01).fit(X2_train,y_train)
LR_model2

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

## Model Evaluation using Test set

In [91]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

#### KNN model evaluation for the two separate sets of independent variables:

In [ ]:
knn_yhat1 = kNN_model1.predict(X1_test)
print("KNN Jaccard index: %.2f" % jaccard_similarity_score(y_test, knn_yhat1))
print("KNN F1-score: %.2f" % f1_score(y_test, knn_yhat1, average='weighted') )

In [105]:
knn_yhat2 = kNN_model2.predict(X2_test)
print("KNN Jaccard index: %.2f" % jaccard_similarity_score(y_test, knn_yhat2))
print("KNN F1-score: %.2f" % f1_score(y_test, knn_yhat2, average='weighted') )

KNN Jaccard index: 0.60
KNN F1-score: 0.60


#### Decision Tree model evaluation for the two separate sets of independent variables:

In [116]:
DT_yhat1 = DT_model1.predict(X1_test)
print("DT Jaccard index: %.2f" % jaccard_similarity_score(y_test, DT_yhat1))
print("DT F1-score: %.2f" % f1_score(y_test, DT_yhat1, average='weighted') )

DT Jaccard index: 0.69
DT F1-score: 0.67


In [117]:
DT_yhat2 = DT_model2.predict(X2_test)
print("DT Jaccard index: %.2f" % jaccard_similarity_score(y_test, DT_yhat2))
print("DT F1-score: %.2f" % f1_score(y_test, DT_yhat2, average='weighted') )

DT Jaccard index: 0.61
DT F1-score: 0.61


#### Support Vector Machine model evaluation for the two separate sets of independent variables:

In [106]:
SVM_yhat1 = SVM_model1.predict(X1_test)
print("SVM Jaccard index: %.2f" % jaccard_similarity_score(y_test, SVM_yhat1))
print("SVM F1-score: %.2f" % f1_score(y_test, SVM_yhat1, average='weighted') )

SVM Jaccard index: 0.71
SVM F1-score: 0.71


In [107]:
SVM_yhat2 = SVM_model2.predict(X2_test)
print("SVM Jaccard index: %.2f" % jaccard_similarity_score(y_test, SVM_yhat2))
print("SVM F1-score: %.2f" % f1_score(y_test, SVM_yhat2, average='weighted') )

SVM Jaccard index: 0.63
SVM F1-score: 0.63


#### Logistic Regression model evaluation for the two separate sets of independent variables:

In [108]:
LR_yhat1 = LR_model1.predict(X1_test)
LR_yhat_prob1 = LR_model1.predict_proba(X1_test)
print("LR Jaccard index: %.2f" % jaccard_similarity_score(y_test, LR_yhat1))
print("LR F1-score: %.2f" % f1_score(y_test, LR_yhat1, average='weighted') )
print("LR LogLoss: %.2f" % log_loss(y_test, LR_yhat_prob1))

LR Jaccard index: 0.72
LR F1-score: 0.71
LR LogLoss: 0.53


In [109]:
LR_yhat2 = LR_model2.predict(X2_test)
LR_yhat_prob2 = LR_model2.predict_proba(X2_test)
print("LR Jaccard index: %.2f" % jaccard_similarity_score(y_test, LR_yhat2))
print("LR F1-score: %.2f" % f1_score(y_test, LR_yhat2, average='weighted') )
print("LR LogLoss: %.2f" % log_loss(y_test, LR_yhat_prob2))

LR Jaccard index: 0.63
LR F1-score: 0.63
LR LogLoss: 0.63
